In [1]:
import openai
import os

os.environ['OPENAI_API_KEY'] = '...'

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"] 

Prolazimo kroz PDF fajl i sa svek stranice izvlacimo tekst. Varijabla tekst na kraju sadrzi sav tekst sa slike.

In [4]:
from PyPDF2 import PdfReader

file_path = 'C:/Users/enaf/Documents/OPENAI/Različite tehnike čunkovanja dokumenata.pdf'
text = ''
# Otvori PDF fajl
with open(file_path, 'rb') as pdf_file:
    pdf_reader =PdfReader(pdf_file)
    for page in pdf_reader.pages:
            text += page.extract_text()

To count the number of tokens that gpt-3.5-turbo will use for some text we need to initialize the tiktoken tokenizer

In [5]:
import tiktoken

tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [6]:
token_counts = tiktoken_len(text)
token_counts

2529

Nas dokument ima ukupno 2529 tokena.

In [7]:
text

' \nRazličite tehnike čunkovanja dokumenata  \n \nŠta su tokeni ? \n \nTokeni  se mogu definisati kao delovi re či. Pre nego što API procesuira prompt, ulaz se razlaže \nna tokene. Ti tokeni su osnovna merna jedinica kojom se računaju troškovi korišćenja \nodređenog OpenAI modela, o čemu će kasnije biti više reči.  \nKako biste stekli bolju predstavu o veličini tokena, naredna pravila objašnjavaju odnos između \ntokena i karaktera/reči u engleskom jeziku : \n• 1 token  ~= 4 karaktera  \n• 1 token ~= ¾ reči  \n• 100 t okena ~= 75 reči  \nSplitovanje teksta u tokene zavisi od jezika. Npr. Cómo estás’ na Španskom sadrži  5 tokena  (za \n10 karaktera ). Zbog toga je implementacija API -a skuplja ako se ne radi o engleskom jeziku.  \nRačunanje tokena na konkretnim primerima možete istr ažiti na linku: \nhttps://platform.openai.com/tokenizer  \nOgraničenja tokena  \n \nTokene treba poznavati, jer različiti modeli imaju različita ograničenja u pogledu maksimalne \ndozvoljene veličine zahteva.

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', '.']
)

In [14]:
chunks = text_splitter.split_text(text)
len(chunks)

8

In [15]:
chunks[0]

'Različite tehnike čunkovanja dokumenata  \n \nŠta su tokeni ? \n \nTokeni  se mogu definisati kao delovi re či. Pre nego što API procesuira prompt, ulaz se razlaže \nna tokene. Ti tokeni su osnovna merna jedinica kojom se računaju troškovi korišćenja \nodređenog OpenAI modela, o čemu će kasnije biti više reči.  \nKako biste stekli bolju predstavu o veličini tokena, naredna pravila objašnjavaju odnos između \ntokena i karaktera/reči u engleskom jeziku : \n• 1 token  ~= 4 karaktera  \n• 1 token ~= ¾ reči  \n• 100 t okena ~= 75 reči  \nSplitovanje teksta u tokene zavisi od jezika. Npr. Cómo estás’ na Španskom sadrži  5 tokena  (za \n10 karaktera ). Zbog toga je implementacija API -a skuplja ako se ne radi o engleskom jeziku.  \nRačunanje tokena na konkretnim primerima možete istr ažiti na linku: \nhttps://platform.openai.com/tokenizer  \nOgraničenja tokena  \n \nTokene treba poznavati, jer različiti modeli imaju različita ograničenja u pogledu maksimalne'

In [16]:
tiktoken_len(chunks[0])

366

In [17]:
chunks[1]

'Ograničenja tokena  \n \nTokene treba poznavati, jer različiti modeli imaju različita ograničenja u pogledu maksimalne \ndozvoljene veličine zahteva. Maksimalni broj tokena za gpt -4 je 8192. M aksimalna veličina \nzahteva za gpt-3.5-turbo   iznosi 4097 tokena, što znači da ulazni prompt i odgovor koji \ndaje OpenAI dele 4097 tokena.  Ovaj problem postaje značaj ukoliko se prosleđuje i istorija \nrazgovora , jer se kroz razgovor količina ulaza u model povećava, jer se svaki put šalje \nkumulativan tog razgovora.  \nMaksimalan broj tokena po za htevu za svaki model možete pronaći na sledećem linku : \nhttps://platform.openai.com/docs/models/  \nCene za 1000 tokena mozete pronaći na: https://openai.com/pricing  \nZa pomoć prilikom računanja cene možete posetiti razne sajtove koje računaju cene za \nOpenAI modele: https://gptforwork.com/tools/openai -chatgpt -api-pricing -calculator  \n \nOptmizacija prompt -a \n \nKako bi prompt bio što kvalitetniji, i sadržao relevantne podatke, razvij

In [18]:
tiktoken_len(chunks[1])

370

In [19]:
chunks[2]

'Optmizacija prompt -a \n \nKako bi prompt bio što kvalitetniji, i sadržao relevantne podatke, razvijene su različite tehnike \nsplitovanja (čankovanja) teksta koj im se prevazilaze ograničenja u pogledu broja tokena.   \nspaCy biblioteka  \n \nspaCy  je popularna open -source Python biblioteka koja se koristi za obradu prirodnog jezika \n(NLP - Natural Language Processing ). Ova biblioteka je razvijena kako bi omogućila brzu i \nefikasnu obradu teksta, uključujući segmentaciju teksta na rečenice i tokene, izdvajanje \nentiteta (imenovanih entiteta kao što su imena, datumi, lokacije), analizu zavisnosti između \nreči i još mnogo toga.  \nspaCy podržava više od  70 jezika. Nažalost, srpski je zik još uvek nije podržan, ali se može \nkoristiti za hrvatski i slovenački.  \nGithub projekta gde se koristi spaCy biblioteka : https://github.com/enaxfa/Chat -with -PDF \n \n \n \n  \n \nTokenizacija  \n \nU dubokom učenju, tokenizacija  je proces pretvaranja niza karaktera u niz tokena koji se 

In [20]:
tiktoken_len(chunks[2])

350

Iako se nekada desi da se prekine chunk u pola recenice i da se izgubi znacenje, sa overlap 
parametrom omogucavamo da pocetak narednog chunka bude 50 tokena iza kraja i da se uhvati semantika isecene recenice.